# Proyecto final para el Laboratorio de Machine Learning - ITBA

## Objetivo: clasificación de artículos periodísticos en categorías

El objetivo de este proyecto es desarrollar una aplicación capaz de tomar un texto periodístico y, en base a su contenido, determinar a qué categoría pertenece. Las categorías elegidas son las siguientes:

* Política
* Economía
* Deportes
* Entretenimiento
* Tecnología


## Dataset: Reuters Corpora (RCV1)

http://trec.nist.gov/data/reuters/reuters.html

El dataset consiste de alrededor de 800.000 artículos periodísticos en formato XML, que van desde 1996-08-20 a 1997-08-19 (un año completo). De cada XML nos interesan los campos: título, cuerpo de la noticia y categorías.

### Preprocesamiento

El preprocesamiento de los archivos de Reuters se realizó con el siguiente script:

[Reuters - preprocessing.ipynb](./Reuters - preprocessing.ipynb)

Del total de artículos se procesan sólo los que pertenecen a las categorías elegidas, con lo cual nos quedamos con un total de aproximadamente 202.400 archivos.

La distribución de cantidad de artículos por categoría queda un poco desbalanceada:
```
 ECONOMICS:     117.539
 ENTERTAINMENT:   3.801
 POLITICS:       56.878
 SPORTS:         35.317
 TECHNOLOGY:      2.410
```

**Nota:** Un artículo puede pertenecer a más de una catagoría, de hecho 13.500 artículos son multi-categoría:
```
 POLITICS_ECONOMICS:        12.948
 ENTERTAINMENT_POLITICS:       272
 ENTERTAINMENT_ECONOMICS:       48
 ENTERTAINMENT_TECHNOLOGY:      40
 POLITICS_TECHNOLOGY:           46
 POLITICS_TECHNOLOGY_ECONOMICS:  6
 SPORTS_ECONOMICS:              39
 SPORTS_ENTERTAINMENT:          15
 SPORTS_POLITICS:               26
 SPORTS_POLITICS_ECONOMICS:      7
 SPORTS_TECHNOLOGY:              1
 SPORTS_ENTERTAINMENT_POLITICS:  1
 TECHNOLOGY_ECONOMICS:          33
 ENTERTAINMENT_TECHNOLOGY_POLITICS: 2
 POLITICS_ENTERTAINMENT_ECONOMICS: 23
```

Ejemplo de XML original: [../rcv1/19960820/4155newsML.xml](./4155newsML.xml)

Ejemplo de TXT convertido (../converted/19960820/4155-POLITICS_ECONOMICS): 

<pre>
California asks U.S. to repay its Civil War debt.
California's state Senate passed a resolution Monday urging President Clinton and the Congress to reimburse the state for expenses it incurred during the Civil War.Republican Assemblyman Mickey Conroy's resolution moved with little fanfare through the state Legislature. It was approved late Monday by the Senate by a vote of 32-to-1. It was backed earlier this year in the state Assembly.The resolution "memorializes" Clinton and Congress to enact legislation that would finally repay the long-forgotten Civil War debt, now estimated to be worth $82 million.Conroy maintains the Civil War debt represents the first unfunded mandate imposed on California by Washington.In the early 1860s, the Congress and the administration of President Abraham Lincoln solicited financial support from the...
</pre>

## Clasificación con CountVectorizer + FNN

La primera idea es ver qué sucede con una simple FNN cuya entrada sea la matriz generada con CountVectorizer.

### CountVectorizing

El siguiente script es el encargado de procesar los artículos y generar la matriz que tendrá una fila por artículo y  columnas que corresponden a la cantidad de palabras encontradas por cada artículo:

[CountVectorizer.ipynb](./CountVectorizer.ipynb)

El countvectorizer utiliza la función `tokenize_and_stem` para tokenizar y reducir la cantidad de token mediante stemming.

El resultado es una matriz sparse de (202399, 84168) que se almacena en `countvect-articles.mtx` para luego ser utilizado en la etaba de entrenamiento de la FNN.

### FNN

La red neuronal más sencilla es simplemente una capa densa con 5 salidas, una por cada una de las categorías que nos interesan. La cantidad de entradas corresponde a las 84K columnas de la cantidad de palabras por artículo, con lo cual queda una red con (84168 * 5) + 5 = 420.845 parámetros. El script de entrenamiento es el siguiente:

[FNN-from-countvectorizer.ipynb](./FNN-from-countvectorizer.ipynb)

El resultado del entrenamiento da a priori un buen resultado con el set de validación, **0.9766**.

Luego del entrenamiento se hicieron pruebas puntuales con textos actuales copiados de sitios de internet con buenos resultados.

También se calculó la precisión con el dataset de test, pero aceptando sólo los resultados de clasificación exactos, es decir, que las 5 categorías dieran el valor de los labels, y dio una precisión de **0.9053**. Siendo más flexibles y aceptando que predijo alguna de las categorías, la presición sube a **0.9473**.

También se realizó un experimento con otra fórmula de loss llamada [kullback_leibler_divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence), pero los resultados no fueron mejores:

[FNN-KL.ipynb](./FNN-KL.ipynb)

**Pendiente**: Hacer pruebas cambiando los valores de *count* por valores normalizados o simplemente un "1" si existe la palabra en la noticia.

## Clasificación con Word2Vec + CNN

Existen varios estudios de NLP en los cuales se aplican redes convolucionales para reducir la cantidad de parámetros necesarios para entrenar la red, uno de los papers es el siguiente:

[Convolutional Neural Networks for Sentence Classification by Yoon Kim](https://arxiv.org/pdf/1408.5882v2.pdf)

La idea consiste en separar el entrenamiento en dos partes: 1) entrenar una matriz de word embedding utilizando  gensim-Word2vec para generar una matriz con el vocabulario propio del dataset de Reuters, y 2) entrenar una red convolucional dejando los pesos fijos de la capa de embedding.

El siguiente esquema explica cómo se arma la red:

<img src="http://www.wildml.com/wp-content/uploads/2015/11/Screen-Shot-2015-11-06-at-12.05.40-PM-1024x937.png"/>

A continuación se explican cada una de las etapas:

### Word2vec

El siguiente script es el encargado de procesar los artículos mediante gensim-word2vec:

[Word2Vec training.ipynb](./Word2Vec training.ipynb)

El entrenamiento con word2vec es sencillo, simplemente hubo que crear un generador capaz de iterar sobre todos los archivos. Se seleccionó un tamaño de vector de 300 de manera que sea compatible con vectores generados por terceros, lo que nos permitirá luego comparar las soluciones aplicando *transfer learning*.

El resultado del entrenamiento es una matriz de 84.000 términos x 300 (tamaño del vector), y que corresponde al primer rectángulo blanco de la imagen anterior. Los pesos de dicha matriz quedan fijos, es decir, no son pesos "entrenables" en la etapa siguiente.

### CNN

La red completa se arma ahora con una primera capa convolucional **1D** formada por filtros de distintos tamaños (3, 4, 5 y 6 palabras de largo) y se agregan 2 de cada tamaño. En la figura corresponden a los primeros rectángulos de colores.

Luego siguen las etapas de MaxPooling1D que reducen los tamaños de la salida de los filtros en 10x. Las salidas luego se hacen "flatten" para pasar a una única dimensión y finalmente se concatenan todas, quedando así listo para entrar a la última capa densa de clasificación.

[CNN-from-word2vec.ipynb](./CNN-from-word2vec.ipynb)

El entrenamiento da como resultado del set de validación **0.9804** y la clasificación de prueba con textos actuales es satisfactoria.





## Transfer Learning utilizando pre-trained Word2vec de Google

Google puso a disposición word embeddings pre entrenados con distintos datos, y en particular el que nos interesa a nosotros es el que surge de un dataset de Google News, que da como resultado una matriz de 1 millón de palabras (case sensitive, con bigrams y trigrams) y con el mismo tamaño de vector de 300. Ver:

[https://code.google.com/archive/p/word2vec/](https://code.google.com/archive/p/word2vec/)

[Google's trained Word2Vec model in Python](http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/)

Se tomó la misma CNN del punto anterior y simplemente se reemplazó el embedding propio por el de Google.

Curiosamente los resultados obtenidos son similares a los anteriores.

**Ventajas:** se ahorra el proceso de entrenamiento de word2vec.

**Desventajas:** se necesita mucha memoria para alojar la matriz de word2vec y un diccionarios para convertir palabras en índices, necesarios para la etapa de entrenamiento de la CNN.




## Clasificación con RNN y word2vec

La idea es armar una simple RNN donde la variable "tiempo" en realidad son las secuencias vectores de las palabras que forman un artículo. La estructura es sencilla:

``` python
model = Sequential()
model.add(w2v_model.wv.get_keras_embedding(train_embeddings=False))
model.add(GRU(100))
model.add(Dense(NUM_CATEGORIES, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
```
<pre>
Layer (type)                 Output Shape              Param #   
=================================================================
embedding_1 (Embedding)      (None, None, 300)         19272000  
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               120300    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 505       
=================================================================
Total params: 19,392,805
Trainable params: 120,805
Non-trainable params: 19,272,000
</pre>

El entrenamiento se hace lento debido a la característica secuencial de las RNN, pero los resultados parecen ser prometedores:

[RNN-word2vec.ipynb](./RNN-word2vec.ipynb)



## Otros experimentos

* [CNN-from-word2vec-trainable.ipynb](./CNN-from-word2vec-trainable.ipynb)


* [W2V-GAP-FC.ipynb](./W2V-GAP-FC.ipynb)


## Conclusiones

* El problema de clasificación "multi label" (la noticia puede pertenecer a más de una categoría) trae algunos desafíos extra con respecto al "multi class", al menos en cuanto a la forma de medir la precisión. Existen algunas propuestas con modelos alternativos que no fueron experimentados: [Introdiction to multi label classification](https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/).


* Tanto la FNN como la CNN dan buenos resultados, pero es posible mejorarlos estudiando mejor las palabras que forman el CountVectorizer y el Word2vec (agregando stopwords, por ejemplo).


* Fue una buena práctica para mejorar el conocimiento de las FNN, CNN, Word2Vec, etc., además de las herramientas de base: Numpy, Keras, etc.
